##  All tables

This code is largely based on the implementation provided by Enge et al. (2021), available at https://osf.io/34ry2/. We are deeply grateful for their dedication to open research.

> Enge, A., Abdel Rahman, R., & Skeide, M. A. (2021). A meta-analysis of fMRI studies of semantic cognition in children. NeuroImage, 241, 118436. https://doi.org/10.1016/j.neuroimage.2021.118436

In [1]:
# Import necessary modules
from os import makedirs, path
from atlasreader import get_statmap_info
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np
import pandas as pd
import seaborn as sns
from nilearn import image, plotting, reporting
from scipy import stats
from scipy.stats import pearsonr
import os, fnmatch
from glob import glob
from pathlib import Path
import re

The Python package you are importing, AtlasReader, is licensed under the
BSD-3 license; however, the atlases it uses are separately licensed under more
restrictive frameworks.
By using AtlasReader, you agree to abide by the license terms of the
individual atlases. Information on these terms can be found online at:
https://github.com/miykael/atlasreader/tree/master/atlasreader/data



In [2]:
def combined_cluster_table(
    img_files_z=[],
    img_files_ale=[],
    stub_keys=[],
    stub_colname="Analysis",
    cluster_extent= 200,
    atlas="aal",
    td_jar=None,
    output_file="",
):

    # Create output director
    output_dir = path.dirname(output_file)
    makedirs(output_dir, exist_ok=True)

    # Create a list of DataFrames with peak and cluster stats for each image
    df_tuples = [
        get_statmap_info(img_file, cluster_extent=cluster_extent, atlas="aal", voxel_thresh=0)
        for img_file in img_files_z
    ]
    dfs = [
        pd.DataFrame(
            {
                "Cluster #": df_tuple[0]["cluster_id"],
                "Size (mm3)": df_tuple[0]["volume_mm"],
                "Cluster labels": df_tuple[0][atlas],
                "Mean z": df_tuple[0]["cluster_mean"],
                "Peak z": df_tuple[1]["peak_value"],
                "Peak X": df_tuple[1]["peak_x"],
                "Peak Y": df_tuple[1]["peak_y"],
                "Peak Z": df_tuple[1]["peak_z"],
                "Peak label": df_tuple[1][atlas],
            }
        )
        for df_tuple in df_tuples
    ]

    # Add ALE values if available
    if img_files_ale:
        df_tuples_ale = [
            get_statmap_info(img_file, cluster_extent=0, atlas="aal", voxel_thresh=0)
            if img_file
            else (
                pd.DataFrame({"cluster_mean": [float("nan")]}),
                pd.DataFrame({"peak_value": [float("nan")]}),
            )
            for img_file in img_files_ale
        ]
        dfs_ale = [
            pd.DataFrame(
                {
                    "Mean ALE": df_tuple[0]["cluster_mean"],
                    "Peak ALE": df_tuple[1]["peak_value"],
                }
            )
            for df_tuple in df_tuples_ale
        ]
        for df, df_ale in zip(dfs, dfs_ale):
            df.insert(4, column="Mean ALE", value=df_ale["Mean ALE"])
            df.insert(6, column="Peak ALE", value=df_ale["Peak ALE"])

    # Concatenate into one big DataFrame
    df = pd.concat(dfs, keys=stub_keys)

    # Reformat numerical columns
    df["Size (mm3)"] = df["Size (mm3)"].apply(lambda x: "{:,.0f}".format(x))
    cols_int = ["Cluster #", "Peak X", "Peak Y", "Peak Z"]
    df[cols_int] = df[cols_int].applymap(int)
    cols_2f = ["Mean z", "Peak z"]
    df[cols_2f] = df[cols_2f].applymap(lambda x: "{:,.2f}".format(x))
    if img_files_ale:
        cols_3f = ["Mean ALE", "Peak ALE"]
        df[cols_3f] = df[cols_3f].applymap(lambda x: "{:,.3f}".format(x))
        df[cols_3f] = df[cols_3f].replace("nan", "")

    # Add the stub column
    df.index = df.index.set_names(stub_colname, level=0)
    df.reset_index(level=stub_colname, inplace=True)
    mask = df[stub_colname].duplicated()
    df.loc[mask.values, [stub_colname]] = ""

    # Save to CSV
    df.to_csv(output_file, sep="\t", index=False)

    return df

In [5]:
print(os.getcwd())


/Users/ss/Documents/Self_Psych_Meta


In [4]:
os.chdir("/Users/ss/Documents/Self_Psych_Meta")

### Tables - ALE results

#### Table 1.  Single analysis: healthy control group

In [6]:
tab1 = combined_cluster_table(
    img_files_z=["/Users/ss/Documents/Self_Psych_Meta/Output/1_ALE/control_all_z_size_level_thresh.nii.gz"],
    cluster_extent=10,
    stub_keys=["Activation likelihood estimation"],
    stub_colname="Analysis",
    img_files_ale=["/Users/ss/Documents/Self_Psych_Meta/Output/1_ALE/control_all_stat_size_thresh.nii.gz", None],
    atlas="aal",
    output_file="/Users/ss/Documents/Self_Psych_Meta/Output/5_Figures/tab_ale_control.tsv"
)

display(tab1)

/Users/ss/miniconda3/lib/python3.12/site-packages/atlasreader/atlasreader.py:496: FutureWarning: From release 0.13.0 onwards, this function will, by default, copy the header of the input image to the output. Currently, the header is reset to the default Nifti1Header. To suppress this warning and use the new behavior, set `copy_header=True`.
  thresh_img = image.threshold_img(stat_img, threshold=voxel_thresh)
/Users/ss/miniconda3/lib/python3.12/site-packages/atlasreader/atlasreader.py:496: FutureWarning: From release 0.13.0 onwards, this function will, by default, copy the header of the input image to the output. Currently, the header is reset to the default Nifti1Header. To suppress this warning and use the new behavior, set `copy_header=True`.
  thresh_img = image.threshold_img(stat_img, threshold=voxel_thresh)
/var/folders/yk/78rqxlxn4pz_rsb5_31xvh340000gn/T/ipykernel_31881/3113370999.py:68: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[cols_i

,Analysis,Cluster #,Size (mm3),Cluster labels,Mean z,Mean ALE,Peak z,Peak ALE,Peak X,Peak Y,Peak Z,Peak label
0,Activation likelihood estimation,1,"16,416",29.87% Cingulate_Ant_L; 28.41% Frontal_Sup_Med...,3.72,0.022,3.72,0.050,0,48,8,Cingulate_Ant_L
1,,2,"4,800",41.83% Frontal_Inf_Orb_2_L; 27.67% OFCpost_L; ...,3.72,0.021,3.72,0.035,-38,24,-14,OFCpost_L
2,,3,"3,256",43.00% Precuneus_L; 30.71% Cingulate_Post_L; 1...,3.72,0.023,3.72,0.043,-2,-54,26,Cingulate_Post_L
3,,4,"1,328",61.45% Cingulate_Mid_L; 38.55% Cingulate_Mid_R,2.73,0.022,2.73,0.036,2,-16,38,Cingulate_Mid_R
4,,5,"1,096",64.23% Angular_L; 35.77% Temporal_Mid_L,2.27,0.022,2.27,0.035,-44,-58,24,Angular_L


#### Table 2.  Single analysis: psychiatric group

In [9]:
# Table 2 -  single analysis - psychiatric group

tab2 = combined_cluster_table(
    img_files_z=["/Users/ss/Documents/Self_Psych_Meta/Output/1_ALE/patient_minus_control_z_size_level_thresh.nii.gz"],
    cluster_extent=10,
    stub_keys=["Activation likelihood estimation"],
    stub_colname="Analysis",
    img_files_ale=["/Users/ss/Documents/Self_Psych_Meta/Output/1_ALE/patient_minus_control_stat_size_thresh.nii.gz", None],
    atlas="aal",
    output_file="Output/5_Figures/tab_ale_patient_minus_control.tsv"
)

display(tab2)

/Users/ss/miniconda3/lib/python3.12/site-packages/atlasreader/atlasreader.py:496: FutureWarning: From release 0.13.0 onwards, this function will, by default, copy the header of the input image to the output. Currently, the header is reset to the default Nifti1Header. To suppress this warning and use the new behavior, set `copy_header=True`.
  thresh_img = image.threshold_img(stat_img, threshold=voxel_thresh)
/Users/ss/miniconda3/lib/python3.12/site-packages/atlasreader/atlasreader.py:496: FutureWarning: From release 0.13.0 onwards, this function will, by default, copy the header of the input image to the output. Currently, the header is reset to the default Nifti1Header. To suppress this warning and use the new behavior, set `copy_header=True`.
  thresh_img = image.threshold_img(stat_img, threshold=voxel_thresh)
/var/folders/yk/78rqxlxn4pz_rsb5_31xvh340000gn/T/ipykernel_51269/3113370999.py:68: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[cols_i

,Analysis,Cluster #,Size (mm3),Cluster labels,Mean z,Mean ALE,Peak z,Peak ALE,Peak X,Peak Y,Peak Z,Peak label


In [3]:
# Table 3 -  single analysis - control minus psychiatric group

tab4 = combined_cluster_table(
    img_files_z=["/Users/ss/Documents/Self_Psych_Meta/Output/1_ALE/patient_z_size_level_thresh.nii.gz"],
    cluster_extent=10,
    stub_keys=["Activation likelihood estimation"],
    stub_colname="Analysis",
    img_files_ale=["/Users/ss/Documents/Self_Psych_Meta/Output/1_ALE/patient_stat_size_thresh.nii.gz", None],
    atlas="aal",
    output_file="/Users/ss/Documents/Self_Psych_Meta/Output/4_Tables/tab_ale_patients.tsv"
)

display(tab4)

/Users/ss/miniconda3/lib/python3.12/site-packages/atlasreader/atlasreader.py:496: FutureWarning: From release 0.13.0 onwards, this function will, by default, copy the header of the input image to the output. Currently, the header is reset to the default Nifti1Header. To suppress this warning and use the new behavior, set `copy_header=True`.
  thresh_img = image.threshold_img(stat_img, threshold=voxel_thresh)
/Users/ss/miniconda3/lib/python3.12/site-packages/atlasreader/atlasreader.py:496: FutureWarning: From release 0.13.0 onwards, this function will, by default, copy the header of the input image to the output. Currently, the header is reset to the default Nifti1Header. To suppress this warning and use the new behavior, set `copy_header=True`.
  thresh_img = image.threshold_img(stat_img, threshold=voxel_thresh)
/var/folders/yk/78rqxlxn4pz_rsb5_31xvh340000gn/T/ipykernel_90447/3113370999.py:68: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[cols_i

,Analysis,Cluster #,Size (mm3),Cluster labels,Mean z,Mean ALE,Peak z,Peak ALE,Peak X,Peak Y,Peak Z,Peak label
0,Activation likelihood estimation,1,"1,520",45.79% Precuneus_L; 28.42% Cingulate_Post_L; 1...,3.01,0.011,3.01,0.017,-2,-54,28,Cingulate_Post_L
1,,2,"1,352",94.08% Frontal_Sup_Medial_L; 5.92% Frontal_Sup...,2.85,0.010,2.85,0.014,-6,58,18,Frontal_Sup_Medial_L


#### Table 3.  Subtraction analysis (conjunction computation results)

Conjunction analysis of all the brain regions that are involved in self-referential processing in both groups

In [ ]:
img_conj_z = "../results/subtraction/control_conj_patients_z.nii.gz"
img_conj_ale = "../results/subtraction/control_conj_patients_ale.nii.gz"
output_file = "../results/tables/tab_conjunction.tsv"

tab3 = combined_cluster_table(
    img_files_z=[img_conj_z],
    cluster_extent=10,
    stub_keys=["Activation likelihood estimation"],
    stub_colname="Analysis",
    img_files_ale=[img_conj_ale, None],
    atlas="harvard_oxford",
    output_file=output_file
)

display(tab3)

/var/folders/yk/78rqxlxn4pz_rsb5_31xvh340000gn/T/ipykernel_61405/3152266981.py:68: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[cols_int] = df[cols_int].applymap(int)
/var/folders/yk/78rqxlxn4pz_rsb5_31xvh340000gn/T/ipykernel_61405/3152266981.py:70: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[cols_2f] = df[cols_2f].applymap(lambda x: "{:,.2f}".format(x))
/var/folders/yk/78rqxlxn4pz_rsb5_31xvh340000gn/T/ipykernel_61405/3152266981.py:73: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[cols_3f] = df[cols_3f].applymap(lambda x: "{:,.3f}".format(x))


,Analysis,Cluster #,Size (mm3),Cluster labels,Mean z,Mean ALE,Peak z,Peak ALE,Peak X,Peak Y,Peak Z,Peak label
0,Activation likelihood estimation,1,872,77.06% Left_Frontal_Pole; 13.76% Left_Paracing...,3.54,3.538,4.31,4.307,-4,58,14,65.0% Left_Frontal_Pole; 15.0% Left_Paracingul...
